In [ ]:

from dotenv import load_dotenv

load_dotenv('../../.env')
import sys
sys.path.insert(0, '../app')
from SIWeaviateClient import SIWeaviateClient
from processing.image.SIImageDescription import SIImageDescription
from processing.text.SIITranslator import SITranslator
from processing.text.SISurya import SISurya
from S3Storage import S3Storage

s3 = S3Storage()
image_descriptor = SIImageDescription()
translator = SITranslator()
surya = SISurya()



## process single pdf

In [ ]:
from processing.PDFProcessor import PDFProcessor
import os
import glob


with SIWeaviateClient() as client:
    pdf_path = "./data/UniversScience/dossiers-pédagogiques/astro/2015/2015 - Dinosaures.pdf"
    pdf = PDFProcessor(client, image_descriptor, translator, surya, s3, pdf_path)



## process all pdfs

In [ ]:
from processing.PDFProcessor import PDFProcessor
import os
import glob

def get_all_pdfs():
    pdf_files = []
    for root, dirs, files in os.walk("./data"):
        for file in files:
            if file.endswith(".pdf"):
                pdf_files.append(os.path.join(root, file))
    return pdf_files


with SIWeaviateClient() as client:
    print("CLIENT", client)
    pdf_paths = get_all_pdfs()
    for pdf_path in pdf_paths[:1]:
        print("==========================================")
        print(pdf_path)
        pdf = PDFProcessor(client, image_descriptor, translator, pdf_path)



# test bugs

In [ ]:
import sys
sys.path.insert(0, '../app')
from processing.image.SIImageDescription import SIImageDescription
image_descriptor = SIImageDescription()

In [ ]:
from IPython.display import Image 
from dotenv import load_dotenv
load_dotenv('../../.env')
import sys
sys.path.insert(0, '../app')

import io
import fitz
from PIL import Image
from pymupdf import Document as PdfDocument

def get_pdf_images(doc: type[PdfDocument]):
    temp_images = []

    for page_num in range(len(doc)):
        page = doc[page_num]

        for item in doc.get_page_images(page_num):
            pix = fitz.Pixmap(doc, item[0])  # pixmap from the image xref
            pix0 = fitz.Pixmap(fitz.csRGB, pix)  # force into RGB
            pil_image = Image.frombytes("RGB", [pix0.width, pix0.height], pix0.samples)
            x0, y0, x1, y1 = page.get_image_bbox(item[7])
            width = x1 - x0
            height = y1 - y0
            # print(f"Width: {width}, Height: {height}")
            # print(f"Image bounding box: x0={x0}, y0={y0}, x1={x1}, y1={y1}")
            # display(pil_image)
            temp_images.append({"image": pil_image, "page_number": page_num+1, 'bbox': {"x0": x0, "y0": y0, "x1": x1, "y1": y1, }})

    return temp_images

# image_descriptor = SIImageDescription()
pdf_path = "/home/erwan/Desktop/clients/ScienceInfuse/server/notebooks/data/UniversScience/revue-decouverte/decouverte_436.pdf"
# pdf_path = "/home/erwan/Desktop/clients/ScienceInfuse/server/notebooks/data/UniversScience/dossiers-pédagogiques/astro/2015/2015 - Dinosaures.pdf"
with fitz.open(pdf_path) as fitz_doc:
    images = get_pdf_images(fitz_doc)
    print(images[0])
    images_chunks = [{**image, "description_en": image_descriptor.get_description(image['image'])} for image in images]

    # for image in images:
    #     display(image)
    # images_chunks = [{**image, "description_en": image_descriptor.get_description(image['image'])} for image in images]


In [ ]:
images_chunks